In [1]:
# import dependendencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
df = pd.read_csv('uscities.csv')


In [3]:
cities = list(df['city'])
state_ids = list(df['state_id'])

In [4]:
mod_cities = []
mod_state_ids = []

for city, state_id in zip(cities, state_ids):
    mod_city = city.replace(' ', '-').lower()
    mod_cities.append(mod_city)
    
    mod_state_id = state_id.lower()
    mod_state_ids.append(mod_state_id)
    

In [5]:
selected_columns = df[['lat', 'lng', 'population']]
uscities_df = selected_columns.copy()
uscities_df['state'] = mod_state_ids
uscities_df['city'] = mod_cities
cols = ['state', 'city', 'lat', 'lng', 'population']
uscities_df = uscities_df[cols]


In [6]:
filtered_df = uscities_df.loc[(uscities_df['state'] == 'ny') | (uscities_df['state'] == 'nj') | (uscities_df['state'] == 'pa')]
filtered_df = filtered_df.sort_values(by=['state']).reset_index(drop = True)


In [7]:
states = ['nj', 'ny', 'pa', 'ct']

state_dfs = []

def create_state_dfs(source_df, states):
   
    for state in states:
        state_df  = source_df.loc[source_df['state'] == state]
        state_df = state_df.sort_values(by=['population'], ascending=False).reset_index(drop = True).head(25)
        state_dfs.append(state_df)
        state_df.empty


create_state_dfs(filtered_df, states) 


# Concatenate the dataframes
top_cities_df = pd.concat(state_dfs, ignore_index=True)


In [8]:
# Open the browser from which we want to extract 
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://www.meetup.com/find/us--ny--new-york/outdoors-adventure"
browser.visit(url)
html = browser.html

In [9]:
soup = bs(html, 'html.parser')

In [10]:
categories = soup.find_all('div', 'css-1q80wuk')
categories

categories_lst = []

for category in categories:
    for a in category.find_all('a', href=True):
        link_text = a['href']
        category_link = link_text.split('/')
        categories_lst.append(category_link[3])
    

categories_lst

['outdoors-adventure',
 'tech',
 'parents-family',
 'health-wellness',
 'sports-fitness',
 'education',
 'photography',
 'food',
 'writing',
 'language',
 'music',
 'movements',
 'lgbtq',
 'film',
 'games-sci-fi',
 'beliefs',
 'arts-culture',
 'book-clubs',
 'dancing',
 'pets',
 'hobbies-crafts',
 'fashion-beauty',
 'social',
 'career-business']

In [11]:
browser.quit()

In [12]:
# construct urls
category_links = []
for index, row in top_cities_df.iterrows():
    for category in categories_lst:
        cat_url = f"https://www.meetup.com/find/us--{row['state']}--{row['city']}/{category}/?eventType=inPerson&distance=tenMiles"
        category_links.append(cat_url)
        
category_links


['https://www.meetup.com/find/us--nj--trenton/outdoors-adventure/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/tech/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/parents-family/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/health-wellness/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/sports-fitness/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/education/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/photography/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/food/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/writing/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/find/us--nj--trenton/language/?eventType=inPerson&distance=tenMiles',
 'https://www.meetup.com/f

In [13]:
# category_links = ['https://www.meetup.com/find/us--ny--new-york/outdoors-adventure/?eventType=inPerson&distance=tenMiles',
#                   'https://www.meetup.com/find/us--nj--trenton/arts-culture/?eventType=inPerson&distance=tenMiles']
events_arr = []
group_names_arr = []
attendees_lst = []
event_links = []
event_dfs = []

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
for link in category_links:
    
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
      
    browser.visit(link)
    html = browser.html
    
    cat_soup = bs(html, 'html.parser')
    
    # Retrieve state
    state = link.split('--')[1]
    
    # Retrieve city
    city = link.split('--')[2].split('/')[0]
    
    # Retrieve events
    events = cat_soup.find_all('p', class_='css-1jy1jkx') 
    for event in events:
        events_arr.append(event.text)
        

    # Retrieve Group names
    events_descriptions = cat_soup.find_all('p', class_='css-ycqk9')
    for desc in events_descriptions:
        desc_text = desc.text
        desc_text = desc_text.split(':')[1]
        group_names_arr.append(desc_text)


    # Retrieve number of attendees
    attendees = cat_soup.find_all('div', class_="css-1dwjarq")
    for attendee_count in attendees:
        attendee_count = attendee_count.text
        attendee_count = attendee_count.split(' ')[0]
        attendees_lst.append(attendee_count)
        
    # Retrieve event link
    events = cat_soup.find_all('div', class_='css-11tad7j')
    for event in events:
        for a in event.find_all('a', href=True):
            link_text = a['href']
            event_links.append(link_text)
            
    event_data = {'state':state,
                  'city': city,
                  'event_name': events_arr,
                  'group_name': group_names_arr,
                  'attendees': attendees_lst,
                  'event_link': event_links}
    
    browser.quit()
    
    df = pd.DataFrame(event_data)
    event_dfs.append(df)
    df.empty
    
    events_arr = []
    group_names_arr = []
    attendees_lst = []
    event_links = []



# print(events_arr)
# print(group_names_arr)   
# print(attendees_lst)
# print(event_links)


In [14]:
final_meetup_df = pd.concat(event_dfs, ignore_index=True)
final_meetup_df


,state,city,event_name,group_name,attendees,event_link
0,nj,trenton,THURSDAY AFTERNOON AEROBIC HIKE,Washington Crossing Park Association of NJ,8,https://www.meetup.com/washingtoncrossingstate...
1,nj,trenton,Fly-In at Ellisdale Fields,NJ Drone User Group,1,https://www.meetup.com/NJ-Dug/events/275376752
2,nj,trenton,SMART RECOVERY at LifeBack,S.M.A.R.T. Recovery at LifeBack,1,https://www.meetup.com/S-M-A-R-T-Recovery-at-L...
3,nj,trenton,Lifeback's Helping Families Affected By Addiction,Lifeback's Helping Families Affected By Addiction,1,https://www.meetup.com/Lifebacks-Helping-Famil...
4,nj,trenton,Flying Solo Support Group,Flying Solo Support Group,1,https://www.meetup.com/meetup-group-uLdxSObt/e...
...,...,...,...,...,...,...
1785,pa,levittown,SMART RECOVERY at LifeBack,S.M.A.R.T. Recovery at LifeBack,1,https://www.meetup.com/S-M-A-R-T-Recovery-at-L...
1786,pa,levittown,Lifeback's Helping Families Affected By Addiction,Lifeback's Helping Families Affected By Addiction,1,https://www.meetup.com/Lifebacks-Helping-Famil...
1787,pa,levittown,Let's meet up and sing!,BucksMont Squires of Song,1,https://www.meetup.com/BucksMont-Squires-of-So...
1788,pa,bloomsburg,Open Ukulele Jam,Danville Ukulele Meetup,1,https://www.meetup.com/Danville-Ukulele-Meetup...


In [17]:
final_meetup_df.to_csv('Final_data.csv',index=False)

In [16]:
# Load the scraped data into a dataframe